In [8]:
import math as m
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt

ImportError: No module named bpy

In [3]:
"""from here: http://stackoverflow.com/questions/9081553/python-scatter-plot-size-and-style-of-the-marker/24567352#24567352"""
def circles(x, y, s, c='b', ax=None, vmin=None, vmax=None, **kwargs):
    """
    Make a scatter of circles plot of x vs y, where x and y are sequence 
    like objects of the same lengths. The size of circles are in data scale.

    Parameters
    ----------
    x,y : scalar or array_like, shape (n, )
        Input data
    s : scalar or array_like, shape (n, ) 
        Radius of circle in data scale (ie. in data unit)
    c : color or sequence of color, optional, default : 'b'
        `c` can be a single color format string, or a sequence of color
        specifications of length `N`, or a sequence of `N` numbers to be
        mapped to colors using the `cmap` and `norm` specified via kwargs.
        Note that `c` should not be a single numeric RGB or
        RGBA sequence because that is indistinguishable from an array of
        values to be colormapped.  `c` can be a 2-D array in which the
        rows are RGB or RGBA, however.
    ax : Axes object, optional, default: None
        Parent axes of the plot. It uses gca() if not specified.
    vmin, vmax : scalar, optional, default: None
        `vmin` and `vmax` are used in conjunction with `norm` to normalize
        luminance data.  If either are `None`, the min and max of the
        color array is used.  (Note if you pass a `norm` instance, your
        settings for `vmin` and `vmax` will be ignored.)

    Returns
    -------
    paths : `~matplotlib.collections.PathCollection`

    Other parameters
    ----------------
    kwargs : `~matplotlib.collections.Collection` properties
        eg. alpha, edgecolors, facecolors, linewidths, linestyles, norm, cmap

    Examples
    --------
    a = np.arange(11)
    circles(a, a, a*0.2, c=a, alpha=0.5, edgecolor='none')

    License
    --------
    This code is under [The BSD 3-Clause License]
    (http://opensource.org/licenses/BSD-3-Clause)
    """
    from matplotlib.patches import Circle
    from matplotlib.collections import PatchCollection
    #import matplotlib.colors as colors

    if ax is None:
        ax = plt.gca()    

    if isinstance(c,basestring):
        color = c     # ie. use colors.colorConverter.to_rgba_array(c)
    else:
        color = None  # use cmap, norm after collection is created
    kwargs.update(color=color)

    if isinstance(x, (int, long, float)):
        patches = [Circle((x, y), s),]
    elif isinstance(s, (int, long, float)):
        patches = [Circle((x_,y_), s) for x_,y_ in zip(x,y)]
    else:
        patches = [Circle((x_,y_), s_) for x_,y_,s_ in zip(x,y,s)]
    collection = PatchCollection(patches, **kwargs)

    if color is None:
        collection.set_array(np.asarray(c))
        if vmin is not None or vmax is not None:
            collection.set_clim(vmin, vmax)

    ax.add_collection(collection)
    return collection

In [4]:
class Coral(object):
    """A class that makes corals with a size, disease state, and location"""
    def __init__(self, diam, dis, x, y):
        self.diam = diam
        self.area = sp.pi*(diam/2.)**2.
        self.dis = dis
        self.x = x
        self.y = y
    
    def is_dis(self):
        if self.dis==1:
            print "Diseased"
        else:
            print "Healthy"
            
c01 = Coral(14,0,3.21,4.445) #I removed 1, because __init__() only takes 5 arguments

In [5]:
class Landscape(object):
    """Inputs/Arguments: (Number of Colonies, SFD parameter 1, SFD parameter 2, Clumpiness Parameter)"""
    """A class that creates landscapes characterized by number of colonies N, size freq distribution, and clumpiness"""
    """Assume that the log(coral.area)~normal(mu,sigma), where mu and sigma are the mean and sd of log(Colony_Length in m)"""
    """Assign a fixed length and width for the landscape"""
    
    reef = [[0, 0], [0, 50], [10,50], [10,0]]

    def __init__(self,nCol, SFDmu, SFDsig, prevalence, clump):
        self.nCol = nCol
        self.SFDmu = SFDmu
        self.SFDsig = SFDsig
        self.clump = clump
        self.prevalence = prevalence
        self.corals = []
   
    def generateColonies(self):
        for i in range(0,self.nCol):
            diam = m.exp(np.random.normal(self.SFDmu,self.SFDsig))
            prev = np.random.binomial(1,self.prevalence)
            x = np.random.random(1)*50   # total landscape is 50m in x
            y = np.random.random(1)*10  # total landscape is 10m in y
            self.corals.append(Coral(diam,prev,x,y))
    def bandTransect(self,st_pt,length,width):
        xmin=st_pt[0]
        xmax=st_pt[0]+length
        ymin=st_pt[1]-width/2
        ymax=st_pt[1]+width/2
        ctcols=0
        ctdis=0
        for i in range(0,self.nCol):
            ctcols+=(xmin<=self.corals[i].x<=xmax)& (ymin<=self.corals[i].y<=ymax)
            ctdis+=(xmin<=self.corals[i].x<=xmax)& (ymin<=self.corals[i].y<=ymax)& self.corals[i].dis==1
        self.band_nCol=ctcols
        self.band_disCol=ctdis
        
    def estTransect(self,st_pt,cc_length,cc_width,dc_length,dc_width):
        st_pt = st_pt
        cc_length = cc_length
        cc_width = cc_width
        cc_xmin=st_pt[0]
        cc_xmax=st_pt[0]+cc_length
        cc_ymin=st_pt[1]-cc_width/2.0
        cc_ymax=st_pt[1]+cc_width/2.0
        dc_xmin=st_pt[0]
        dc_xmax=st_pt[0]+dc_length
        dc_ymin=st_pt[1]-dc_width/2.0
        dc_ymax=st_pt[1]+dc_width/2.0
        
        ctcols=0
        ctdis=0
        for i in range(0,self.nCol):
            ctcols+=(cc_xmin<=self.corals[i].x<=cc_xmax)& (cc_ymin<=self.corals[i].y<=cc_ymax)
            ctdis+=(dc_xmin<=self.corals[i].x<=dc_xmax)& (dc_ymin<=self.corals[i].y<=dc_ymax)& self.corals[i].dis==1
        self.estband_nCol=ctcols
        self.estband_disCol=ctdis    
    
    def LI(self,st_pt,length): ##Incorrect Line intercept method. If you don't define a width, python will count all colonies in entire 25 x 10m belt
        xmin=st_pt[0]
        xmax=st_pt[0]+length
        
        ctcols = 0
        ctdis = 0
        for i in range(0,self.nCol):
            ctcols += (xmin<=self.corals[i].x)& (self.corals[i].x<=xmax)
            ctdis += (xmin<=self.corals[i].x<=xmax) & self.corals[i].dis==1
        self.LI_nCol=ctcols
        self.LI_disCol=ctdis
   
    def LI2(self,st_pt,length,width):#revised line intercept method with very small width
        xmin=st_pt[0]
        xmax=st_pt[0]+length
        ymin=st_pt[1]-width/2.0
        ymax=st_pt[1]+width/2.0
        ctcols=0
        ctdis=0
        for i in range(0,self.nCol):
            ctcols+=(xmin<=self.corals[i].x<=xmax)& (ymin<=self.corals[i].y<=ymax)
            ctdis+=(xmin<=self.corals[i].x<=xmax)& (ymin<=self.corals[i].y<=ymax)& self.corals[i].dis==1
        
        self.LI2_nCol=ctcols
        self.LI2_disCol=ctdis
"""Identifying corals that overlap with line. Still working on this code"""       
    def closest_point_on_seg(self,st_pt,end_pt,length):
        seg_v = end_pt - st_pt        
        for i in range(0,self.nCol):
            pt_v[i] = (self.corals[i].x,self.corals[i].y)-(st_pt)
        seg_v_unit = seg_v / self.length()
        
        proj = pt_v.dot(seg_v_unit)###code from website, stopped modifying here
        if proj <= 0:
            return seg_a.copy()
        if proj >= seg_v.len():
            return seg_b.copy()
        proj_v = seg_v_unit * proj
        closest = proj_v + seg_a
        return closest

    def segment_circle(seg_a, seg_b, circ_pos, circ_rad):
        closest = closest_point_on_seg(seg_a, seg_b, circ_pos)
        dist_v = circ_pos - closest
        if dist_v.len() > circ_rad:
            return vec(0, 0)
        if dist_v.len() <= 0:
            raise ValueError, "Circle's center is exactly on segment"
        offset = dist_v / dist_v.len() * (circ_rad - dist_v.len())
        return offset
        
        
        
L01 = Landscape(1000,.18,.2,0.2,0)
L01.generateColonies()
L01.bandTransect([5,5],25,2)
L01.estTransect([5,5],10,1,25,4)
L01.LI([5,5],25)
L01.LI2([5,5],25,0.01)



print L01.band_nCol
print L01.band_disCol

print L01.estband_nCol
print L01.estband_disCol

print L01.LI_nCol
print L01.LI_disCol

print L01.LI2_nCol
print L01.LI2_disCol



L01x = []
L01y = []
L01area = []
L01dis = []
for i in range(0,100):
    L01x.append(L01.corals[i].x)
    L01y.append(L01.corals[i].y)
    L01area.append(L01.corals[i].area*10)
    L01dis.append(L01.corals[i].dis+1)

plt.scatter(L01x,L01y,s=L01area,c = L01dis)

 
   
            

[98]
[17]
[23]
[39]
[474]
[88]
0
0


In [6]:
"""generate a Landscape and plot it"""
nc = 1000
L01 = Landscape(nc,-2.25,1.07,0.2,0)  #mean(log(ColonyLength in m)) = -2.25; sd(log(ColonyLength in m)) = 1.07
L01.generateColonies()

L01x = np.zeros(nc,float)
L01y = np.zeros(nc,float)
L01r = np.zeros(nc,float)
L01dis = np.zeros(nc,int)
for i in range(0,nc):
    L01x[i] = L01.corals[i].x
    L01y[i] = L01.corals[i].y
    L01r[i] = L01.corals[i].diam/2
    L01dis[i] = L01.corals[i].dis+1

from matplotlib.ticker import FixedLocator, FormatStrFormatter
minorLocatorx = FixedLocator(range(0,51,1))
minorLocatory = FixedLocator(range(0,11,1))

from matplotlib.patches import Rectangle

plt.figure(figsize=[20,4])
ax = gca()
circles(L01x,L01y,L01r, edgecolor = 'b', facecolor='none')
Rectangle([5,4],25,2)
plt.xlim([0,50])
plt.ylim([0,10])
ax.xaxis.set_minor_locator(minorLocatorx)
ax.yaxis.set_minor_locator(minorLocatory)
plt.grid(b=True,which='both',axis='both')


plt.figure()
plt.hist(np.multiply(L01r,2),30)
plt.xlim([0,1.5])
plt.xlabel('Colony Diameter in meters')


NameError: name 'gca' is not defined